In [29]:
import os
import json
import csv

import pandas as pd

In [14]:
### dataset info and constants
file_dir = "china_isd_lite_2020"
# list head
year = 0
month = 1
day = 2
hour = 3
air_temperature = 4
dew_point_temperature = 5
sea_level_pressure = 6
wind_direction = 7
wind_speed_rate = 8
sky_condition_total_coverage_code = 9
### sky_condition_total_coverage_code
## DOMAIN:
#  0: None, SKC or CLR
#  1: One okta - 1/10 or less but not zero
#  2: Two oktas - 2/10 - 3/10, or FEW
#  3: Three oktas - 4/10
#  4: Four oktas - 5/10, or SCT
#  5: Five oktas - 6/10
#  6: Six oktas - 7/10 - 8/10
#  7: Seven oktas - 9/10 or more but not 10/10, or BKN
#  8: Eight oktas - 10/10, or OVC
#  9: Sky obscured, or cloud amount cannot be estimated
# 10: Partial obscuration
# 11: Thin scattered
# 12: Scattered
# 13: Dark scattered
# 14: Thin broken
# 15: Broken
# 16: Dark broken
# 17: Thin overcast
# 18: Overcast
# 19: Dark overcast
liquid_precipitation_pepth_dimension_1_hour = 10
liquid_precipitation_pepth_dimension_6_hour = 11

MISSING_VALUE = "-9999"

In [15]:
selected_area = air_temperature

In [51]:
# step 1: read total stations' file

total_stations = {}
count_missing_total = 0
for file_name in os.listdir(file_dir):
  # obtain site_id by file_name
  station_id = file_name.split("-")[0]

  file_object = open(file_dir + "/"+file_name, 'r')
  station_val = []
  station_missing_index = []
  station_missing_count = 0
  line_index = 0


  while(True):
    line = file_object.readline().split()
    if(line):
      if (line[selected_area]==MISSING_VALUE):
        count_missing_total += 1
        station_missing_count+=1
        station_missing_index.append(line_index)
      station_val.append(int(line[selected_area]))
    else: 
      break
    line_index+=1

  
  total_stations[station_id] = {
    "val": station_val,
    "missing_count": station_missing_count,
    "missing_index": station_missing_index,
  }

  # next line output saved in missing count.txt
  # print(site_id, site_missing_count, site_missing_index)

# print(count_missing_total)
# out_file = open("step1.json", "w")
# json.dump(total_stations, out_file)
# out_file.close()

### statistics
# 415 sites
# missing_value_count: 2306
# each site missing saved in missing count.txt


In [ ]:
# step 2: read stations' locations,  site_id-> [lat, lon] and station name

df=pd.read_csv('isd-history.csv')
df2 = df[['USAF', 'STATION NAME', 'LAT', 'LON']]

staion_info_dict = {}
valid_count = 0
total_count=0
for i, j in df2.iterrows():
  total_count+=1
  valid = (not pd.isnull(j['STATION NAME'])) & (
      not pd.isnull(j['LON'])) & (not pd.isnull(j['LAT']))
  if (valid):
    valid_count+=1
    staion_info_dict[j['USAF']]={
        'name': j['STATION NAME'],
        'lon': j['LON'],
        'lat': j['LAT']
    }

print(total_count, valid_count)
# out_file = open("step2.json", "w")
# json.dump(staion_info_dict, out_file)
# out_file.close()

### statistics
# 29562 stations
# valid: 28374


In [ ]:
# step 3: check china stations missing info -> all valid, filter station_info
count = 0
for station_id in total_stations:
  if (station_id not in staion_info_dict) :
    count+=1
# print(count)
# 0

china_station_dict = {}
for station_id in total_stations:
  if (station_id in staion_info_dict) :
    china_station_dict[station_id] = staion_info_dict[station_id]

# print(china_station_dict)
# out_file = open("step3.json", "w")
# json.dump(china_station_dict, out_file)
# out_file.close()


In [ ]:
# step 4: a function to display knn
